In [2]:
import numpy as np
import pandas as pd

In [3]:
train=pd.read_csv("train.csv",index_col=0)
test=pd.read_csv("test.csv",index_col=0)

In [11]:
# 0값을 결측치로 바꾸기

# train
train_dst = train.filter(regex='_dst$', axis=1).replace(0, np.NaN)
train_src = train.filter(regex='_src$', axis=1).replace(0, np.NaN)
# test
test_dst = test.filter(regex='_dst$', axis=1).replace(0, np.NaN)
test_src = test.filter(regex='_src$', axis=1).replace(0, np.NaN)

In [58]:
#보간, 보간되지 않은 값(결측치)는 0으로

# train
train_dst = train_dst.interpolate(methods='linear', axis=1)
train_src = train_src.interpolate(methods='linear', axis=1)
train_dst.fillna(0, inplace=True) 
train_src.fillna(0, inplace=True) 


# test
test_dst = test_dst.interpolate(methods='linear', axis=1)
test_src = test_src.interpolate(methods='linear', axis=1)
test_dst.fillna(0, inplace=True)
test_src.fillna(0, inplace=True)

In [63]:
train_X=pd.merge(train_src,train_dst,on="id")
train_X["rho"]=train["rho"]
x_col=train_X.columns

test_X=pd.merge(test_src,test_dst,on="id")
test_X["rho"]=test["rho"]
X_col=test_X.columns

In [74]:
train_x=train_X[[i for i in x_col]].as_matrix()
y_col=train.columns[-4:]
train_y=train[[i for i in y_col]].as_matrix()

test_x=test_X[[i for i in x_col]].as_matrix()

C:\anaconda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
C:\anaconda\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\anaconda\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


In [50]:
import tensorflow as tf
from tensorflow.keras import layers

In [51]:
def dnn(x,y):
    model = tf.keras.Sequential()  #순차적 계층화 준비
    model.add(layers.Dense(140, input_shape=(71,))) 
    model.add(layers.Activation('relu')) 

    model.add(layers.Dense(140))  
    model.add(layers.Activation('relu'))

    model.add(layers.Dense(140))  
    model.add(layers.Activation('relu'))
    
    model.add(layers.Dense(4))

    # 모델 구축하기
    model.compile(
        loss='mse',
        optimizer="adam",
        metrics=['mae'])
    
    hist = model.fit(
    x, y,
    batch_size=100,  #100개에 한 번씩 업데이터 실행
    epochs=800,     
    validation_split=0.2,  
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)],  
      #'val_loss'를 monitor하여 감소하면 한 번 더(1) 참고 조기중지
    verbose=1)
    
    return hist,model


In [54]:
hist,model=dnn(x,y)

Train on 8000 samples, validate on 2000 samples
Epoch 1/800
8000/8000 [==============================] - 3s 333us/sample - loss: 10.1768 - mae: 2.3800 - val_loss: 7.7067 - val_mae: 2.0835
Epoch 2/800
8000/8000 [==============================] - 0s 40us/sample - loss: 6.8631 - mae: 1.9669 - val_loss: 6.2994 - val_mae: 1.8839
Epoch 3/800
8000/8000 [==============================] - 0s 32us/sample - loss: 5.9143 - mae: 1.8274 - val_loss: 5.8568 - val_mae: 1.8169
Epoch 4/800
8000/8000 [==============================] - 0s 34us/sample - loss: 5.7392 - mae: 1.8005 - val_loss: 5.8499 - val_mae: 1.8170
Epoch 5/800
8000/8000 [==============================] - 0s 34us/sample - loss: 5.6938 - mae: 1.7951 - val_loss: 5.8004 - val_mae: 1.8101
Epoch 6/800
8000/8000 [==============================] - 0s 34us/sample - loss: 5.7303 - mae: 1.8003 - val_loss: 6.0941 - val_mae: 1.8554
Epoch 7/800
8000/8000 [==============================] - 0s 32us/sample - loss: 5.6549 - mae: 1.7883 - val_loss: 5.7744 - 

8000/8000 [==============================] - 0s 54us/sample - loss: 5.5335 - mae: 1.7691 - val_loss: 5.7445 - val_mae: 1.7997
Epoch 60/800
8000/8000 [==============================] - 0s 35us/sample - loss: 5.5617 - mae: 1.7734 - val_loss: 5.7389 - val_mae: 1.7955
Epoch 61/800
8000/8000 [==============================] - 0s 33us/sample - loss: 5.5608 - mae: 1.7739 - val_loss: 5.7261 - val_mae: 1.7966
Epoch 62/800
8000/8000 [==============================] - 0s 38us/sample - loss: 5.5564 - mae: 1.7740 - val_loss: 5.7577 - val_mae: 1.8043
Epoch 63/800
8000/8000 [==============================] - 0s 30us/sample - loss: 5.5341 - mae: 1.7705 - val_loss: 5.7586 - val_mae: 1.8033
Epoch 64/800
8000/8000 [==============================] - 0s 33us/sample - loss: 5.5423 - mae: 1.7708 - val_loss: 5.7164 - val_mae: 1.7951
Epoch 65/800
8000/8000 [==============================] - 0s 32us/sample - loss: 5.5258 - mae: 1.7669 - val_loss: 5.7383 - val_mae: 1.7980
Epoch 66/800
8000/8000 [================

In [78]:
y_pred = model.predict(test_x)
print(y_pred)

[[8.146208  3.9374049 8.94915   2.9461067]
 [8.024466  3.9235132 8.877751  2.8748875]
 [7.753181  3.9041235 8.95191   2.8278854]
 ...
 [7.842301  3.8179266 8.924166  2.7528546]
 [7.5813203 3.8894687 8.400471  2.79084  ]
 [7.994288  4.0006204 9.293969  2.8989902]]


In [79]:
df=pd.DataFrame(columns=["hhb","hbo2","ca","na"])

for i in range(10000):
    df=df.append({"hhb":y_pred[i][0],"hbo2":y_pred[i][1],
                  "ca":y_pred[i][2],"na":y_pred[i][3]},ignore_index=True)

In [80]:
df.to_csv("myfile3.csv",mode="w",index=False)